In [1]:
import pandas as pd
import numpy as np
from astropy.table import Table
import glob
import os

# Find all CSV files in the current directory
csv_files = glob.glob('./*_Decay.csv')

for csv_file in csv_files:
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Get unique combinations of profile and channel
    profiles = df['profile'].unique()
    channels = df['ch'].unique()
    
    # Process each combination
    for profile in profiles:
        for channel in channels:
            # Filter for current profile and channel
            mask = (df['profile'] == profile) & (df['ch'] == channel)
            df_filtered = df[mask]
            
            # Skip if no data for this combination
            if df_filtered.empty:
                continue
            
            # Create a new dataframe with the desired quantiles
            result = pd.DataFrame()
            result['mass'] = df_filtered[df_filtered['quantile'] == 0.5]['mass'].values
            result['tau_50'] = df_filtered[df_filtered['quantile'] == 0.5]['tau'].values
            result['tau_32'] = df_filtered[df_filtered['quantile'] == 0.32]['tau'].values
            result['tau_68'] = df_filtered[df_filtered['quantile'] == 0.68]['tau'].values
            result['tau_05'] = df_filtered[df_filtered['quantile'] == 0.05]['tau'].values
            result['tau_95'] = df_filtered[df_filtered['quantile'] == 0.95]['tau'].values
            
            # Convert to astropy table
            table = Table.from_pandas(result)

             # Set format for all tau columns to show 4 significant figures
            for col in table.colnames:
                if col.startswith('tau'):
                    table[col].format = '.4g'
            
            # Create output filename based on input filename
            # Example: 'CBe_Einasto_Decay.csv' -> 'svlim_CBe_Einasto_b_decay.ecsv'
            base_name = os.path.basename(csv_file)
            source_name = base_name.split('_')[0]  # Gets 'CBe' from 'CBe_Einasto_Decay.csv'
            output_name = f'svlim_{source_name}_{profile}_{channel}_decay.ecsv'
            
            # Save as ECSV
            table.write(output_name, format='ascii.ecsv', overwrite=True)